<a href="https://colab.research.google.com/github/BingHungLee/EDC_Cracking_Furnace_Design/blob/main/%5B20210709%5D_Cantera(L2)_Working_With_Mechanism_Files_%26_Chemical_Equilibrium_%26_Chemical_Kinetics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
################################################################################
# INSTALL CONDA AND CANTERA ON GOOGLE COLAB
################################################################################
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

!conda install --channel cantera cantera ipython matplotlib jupyter

--2021-07-09 06:27:57--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   188MB/s    in 0.4s    

2021-07-09 06:27:57 (188 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h

In [2]:
import cantera as ct
import numpy as np

# Working With Mechanism Files
reference: https://cantera.org/tutorials/python-tutorial.html

Several other reaction mechanism files are included with Cantera, including ones that model high- temperature air, a hydrogen/oxygen reaction mechanism, and a few surface reaction mechanisms. These files are usually located in the data subdirectory of the Cantera installation directory, for example C:\\Program Files\\Cantera\\data on Windows or /usr/local/cantera/data/ on Unix/Linux/Mac OS X machines, depending on how you installed Cantera and the options you specified.

If for some reason Cantera has difficulty finding where these files are on your system, set environment variable CANTERA_DATA to the directory or directories (separated using ; on Windows or : on other operating systems) where they are located. Alternatively, you can call function add_directory to add a directory to the Cantera search path:

In [3]:
ct.add_directory('~/cantera/my_data_files')

Cantera input files are plain text files, and can be created with any text editor.

A Cantera input file may contain more than one phase specification, or may contain specifications of interfaces (surfaces). Here we import definitions of two bulk phases and the interface between them from file diamond.yaml:

In [5]:
gas2 = ct.Solution('diamond.yaml', 'gas')
gas2()


  gas:

       temperature   1200 K
          pressure   2666.4 Pa
           density   0.00057641 kg/m^3
  mean mol. weight   2.1568 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy        1.2418e+07        2.6783e+07  J
   internal energy         7.792e+06        1.6806e+07  J
           entropy             94362        2.0352e+05  J/K
    Gibbs function       -1.0082e+08       -2.1744e+08  J
 heat capacity c_p             14600             31490  J/K
 heat capacity c_v             10745             23176  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                 H        0.00093452         0.0019996           -3.4029
                H2            0.9233            0.9878           -21.627
               CH3         0.0013939        0.00019996           -24.284
 

In [6]:
diamond = ct.Solution('diamond.yaml', 'diamond')
diamond()


  diamond:

       temperature   298.15 K
          pressure   1.0132e+05 Pa
           density   3520 kg/m^3
  mean mol. weight   12.011 kg/kmol
   phase of matter   unspecified

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy                 0                 0  J
   internal energy           -28.786           -345.74  J
           entropy                 0                 0  J/K
    Gibbs function                 0                 0  J
 heat capacity c_p                 0                 0  J/K
 heat capacity c_v                 0                 0  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
              C(d)                 1                 1                 0



In [7]:
diamond_surf = ct.Interface('diamond.yaml' , 'diamond_100', [gas2, diamond])
diamond_surf()


  diamond_100:

       temperature   1200 K
          pressure   1.0132e+05 Pa
           density   5.7456e-08 kg/m^3
  mean mol. weight   1.9152 kg/kmol
   phase of matter   unspecified

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy        3.3709e+07        6.4559e+07  J
   internal energy        3.3709e+07        6.4559e+07  J
           entropy             46961             89939  J/K
    Gibbs function       -2.2644e+07       -4.3368e+07  J
 heat capacity c_p                 0                 0  J/K
 heat capacity c_v                 0                 0  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
              c6HH           0.94737               0.9           -5.8924
              c6H*          0.052632               0.1             9.565
     [   +6 minor]                 0                 0

# Chemical Equilibrium
To set a gas mixture to a state of chemical equilibrium, use the equilibrate method:

In [8]:
import cantera as ct
g = ct.Solution('gri30.yaml')
g.TPX = 300.0, ct.one_atm, 'CH4:0.95,O2:2,N2:7.52'
g.equilibrate('TP')
g()


  gri30:

       temperature   300 K
          pressure   1.0133e+05 Pa
           density   1.1248 kg/m^3
  mean mol. weight   27.689 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -2.8723e+06       -7.9532e+07  J
   internal energy       -2.9624e+06       -8.2026e+07  J
           entropy            7226.6         2.001e+05  J/K
    Gibbs function       -5.0403e+06       -1.3956e+08  J
 heat capacity c_p            1106.5             30638  J/K
 heat capacity c_v            806.22             22323  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                O2          0.011038         0.0095511           -29.325
               H2O           0.11807           0.18147           -121.37
               CO2           0.14422          0.090735           -185.87


the specific enthalpy and pressure can be held fixed:

In [9]:
g.TPX = 300.0, ct.one_atm, 'CH4:0.95,O2:2,N2:7.52'
g.equilibrate('HP')

Other options are:

* UV fixed specific internal energy and specific volume
* SV fixed specific entropy and specific volume
* SP fixed specific entropy and pressure

How can you tell if equilibrate has correctly found the chemical equilibrium state? One way is verify that the net rates of progress of all reversible reactions are zero. Here is the code to do this:

In [10]:
g.TPX = 300.0, ct.one_atm, 'CH4:0.95,O2:2,N2:7.52'
g.equilibrate('HP')

rf = g.forward_rates_of_progress
rr = g.reverse_rates_of_progress
for i in range(g.n_reactions):
  if g.is_reversible(i) and rf[i] != 0.0:
    print(' %4i  %10.4g  ' % (i, (rf[i] - rr[i])/rf[i]))

    0           0  
    1  -1.666e-15  
    2  -5.904e-16  
    3  -7.293e-15  
    4  -1.893e-15  
    5   1.503e-15  
    6   8.915e-15  
    7           0  
    8   5.431e-15  
    9  -3.808e-15  
   10   5.238e-15  
   11           0  
   12   -2.15e-16  
   13   1.505e-15  
   14  -3.435e-15  
   15  -7.457e-15  
   16  -3.312e-15  
   17  -8.898e-15  
   18  -6.361e-15  
   19  -3.765e-15  
   20  -2.766e-15  
   21  -1.421e-15  
   22  -7.336e-15  
   23   1.968e-14  
   24   1.247e-14  
   25   1.043e-14  
   26  -1.057e-14  
   27    1.44e-15  
   28   -7.34e-15  
   29  -1.104e-14  
   30   9.262e-15  
   31  -2.375e-15  
   32  -1.493e-15  
   33   5.654e-15  
   34   5.617e-15  
   35  -7.621e-15  
   37  -5.043e-15  
   38  -3.131e-15  
   39  -3.315e-15  
   40   -3.23e-15  
   41  -3.492e-15  
   42    2.19e-15  
   43           0  
   44  -7.483e-15  
   45  -5.518e-15  
   46  -8.064e-15  
   47  -1.937e-15  
   48  -4.556e-15  
   49   9.261e-15  
   50   1.151e-15  


If the magnitudes of the numbers in this list are all very small, then each reversible reaction is very nearly equilibrated, which only occurs if the gas is in chemical equilibrium.

You might be wondering how equilibrate works. Method equilibrate invokes **Cantera's chemical equilibrium solver**, which uses an e**lement potential method**. The element potential method is one of a **class of equivalent nonstoichiometric methods** that all have the characteristic that the problem reduces **to solving a set of MM nonlinear algebraic equations**, where MM is the number of elements (not species). The so-called stoichiometric methods, on the other hand, (including Gibbs minimization), require solving KK nonlinear equations, where KK is the number of species (usually K >> MK>>M). See Smith and Missen, "Chemical Reaction Equilibrium Analysis" for more information on the various algorithms and their characteristics.

Cantera uses a damped Newton method to solve these equations, and does a few other things to generate a good starting guess and to produce a reasonably robust algorithm. If you want to know more about the details, look at the C++ code in **ChemEquil.h.**

# Chemical Kinetics

Solution() objects are also Kinetics() objects, and provide all of the methods necessary to compute the thermodynamic quantities associated with each reaction, reaction rates, and species creation and destruction rates. They also provide methods to inspect the quantities that define each reaction such as the rate constants and the stoichiometric coefficients. The rate calculation functions are used extensively within Cantera's reactor network model and 1D flame model.

In [13]:
g = ct.Solution('gri30.yaml')
r = g.reaction(2) # get a Reaction object
r

<ElementaryReaction: H2 + O <=> H + OH>

In [14]:
r.reactants

{'H2': 1.0, 'O': 1.0}

In [15]:
r.products

{'H': 1.0, 'OH': 1.0}

In [16]:
r.rate

Arrhenius(A=38.7, b=2.7, E=2.61918e+07)

If we are interested in only certain types of reactions, we can use this information to filter the full list of reactions to find the just the ones of interest. For example, here we find the indices of just those reactions which convert CO into CO2:

In [19]:
II = [i for i,r in enumerate(g.reactions())
if 'CO' in r.reactants and 'CO2' in r.products]

In [20]:
for i in II:
  print(g.reaction(i).equation)

CO + O (+M) <=> CO2 (+M)
CO + O2 <=> CO2 + O
CO + OH <=> CO2 + H
CO + HO2 <=> CO2 + OH


(Actually, we should also include reactions where the reaction is written such that CO2 is a reactant and CO is a product, but for this example, we'll just stick to this smaller set of reactions.) Now, let's set the composition to an interesting equilibrium state:

In [21]:
g.TPX = 300, 101325, {'CH4':0.6, 'O2':1.0, 'N2':3.76}
g.equilibrate('HP')

We can verify that this is an equilibrium state by seeing that the net reaction rates are essentially zero:

In [23]:
g.net_rates_of_progress[II]

array([-1.28749008e-19, -6.35274710e-21, -2.33146835e-15, -8.97854924e-20])

Now, let's see what happens if we decrease the temperature of the mixture:

In [24]:
g.TP = g.T-100, None
g.net_rates_of_progress[II]

array([3.18644907e-05, 5.00489883e-08, 1.05964910e-01, 2.89502678e-06])

All of the reaction rates are positive, favoring the formation of CO2 from CO, with the third reaction, CO + OH <=> CO2 + H proceeding the fastest. If we look at the enthalpy change associated with each of these reactions:

In [25]:
g.delta_enthalpy[II]

array([-5.33034690e+08, -2.23248529e+07, -8.76650141e+07, -2.49169644e+08])

we see that the change is negative in each case, indicating a net release of thermal energy. The total heat release rate can be computed either from the reaction rates:

In [26]:
np.dot(g.net_rates_of_progress, g.delta_enthalpy)

-58013369.598157085

or from the species production rates:

In [27]:
np.dot(g.net_production_rates, g.partial_molar_enthalpies)

-58013369.598157145

The contribution from just the selected reactions is:

In [28]:
np.dot(g.net_rates_of_progress[II], g.delta_enthalpy[II])

-9307122.69279439

Or about 16% of the total heat release rate.